# ヤコビアン

In [1]:
import sympy as sy
from sympy.printing.numpy import NumPyPrinter
from sympy import julia_code
from sympy.utilities.codegen import codegen
import tqdm
import os
from pathlib import Path

from kinematics import Global

In [2]:
kinem = Global(N=3)
kinem.set_J_v_simple()

computing kinematics...
Matrix([[q_large[0, 0]], [q_large[1, 0]], [q_large[2, 0]]])
Matrix([[q_large[3, 0]], [q_large[4, 0]], [q_large[5, 0]]])
Matrix([[q_large[6, 0]], [q_large[7, 0]], [q_large[8, 0]]])
done computing kinematics!


ヤコビアンを時間微分

コード生成

In [4]:
# Cコード生成
cwd = str(Path().resolve())
base = cwd + "/derived/N_is_" + str(kinem.N)
dir_name = base + '/c_src/'
os.makedirs(dir_name, exist_ok=True)


def gen_c(f, name, dir_name):
    [(c_name, c_code), (h_name, c_header)] = codegen(
        name_expr=(name, f),
        language="C",
        project= name + "project",
        to_files=False
    )
    
    f = open(dir_name + c_name, 'w')
    f.write(c_code)
    f.close()

    f = open(dir_name + h_name, 'w')
    f.write(c_header)
    f.close()


os.makedirs(dir_name + "Phi_s/", exist_ok=True)
for i, Phi in enumerate(kinem.Phi_s):
    gen_c(Phi, "Phi_"+str(i), dir_name + "Phi_s/")


os.makedirs(dir_name + "J_s/", exist_ok=True)
for i, J in enumerate(kinem.J_v_simples):
    gen_c(J, "J_"+str(i), dir_name + "J_s/")

